In [1]:
import locale
locale.getpreferredencoding()
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [3]:
!apt update
!apt install -y libsm6 libxext6 libxrender-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [46.6 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.8 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,419 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadc

In [4]:
!pip install trimesh
!pip install open3d
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.2/687.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.5/420.5 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.6 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5

In [5]:
!git clone https://github.com/grapergrape/AMS_izziv.git

Cloning into 'AMS_izziv'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 65 (delta 20), reused 52 (delta 10), pack-reused 0
Receiving objects: 100% (65/65), 6.68 MiB | 12.81 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [6]:
import trimesh
import numpy as np
import cupy as cp
import open3d
import sklearn
from sklearn.neighbors import KNeighborsClassifier

# Load the mesh
mesh = trimesh.load_mesh('/content/AMS_izziv/aneurysms/ru_22_CTA_PT00014_20200906.obj')
mesh.show()

In [7]:
import numpy as np

# Create a copy of the original mesh
deformed_mesh = mesh.copy()

# Get the vertices of the mesh
vertices = deformed_mesh.vertices

# Define a bounding box for one of the bunny's ears
ear_min = np.array([-0.05, 0.15, 0])
ear_max = np.array([0.05, 0.3, 0.1])

# Select vertices that fall within the bounding box
ear_vertices = ((vertices > ear_min) & (vertices < ear_max)).all(axis=1)

# Apply a non-linear transformation to the selected ear vertices
vertices[ear_vertices, 1] = vertices[ear_vertices, 1] + 0.05 * np.sin(10 * vertices[ear_vertices, 0])

# Replace the vertices in the deformed mesh
deformed_mesh.vertices = vertices

deformed_mesh.show()


# Coherent Point Drift implementation

In [8]:
def simplify_mesh(mesh, steps):
    """
    Simplify a 3D mesh by reducing the number of faces.

    Args:
        mesh (trimesh.Trimesh): The input 3D mesh.
        steps (int): The number of simplification steps.

    Returns:
        trimesh.Trimesh: The simplified mesh.
    """
    for _ in range(steps):
        # Apply quadratic decimation to reduce the face count by half in each step.
        mesh = mesh.simplify_quadric_decimation(mesh.faces.shape[0] // 2)
    return mesh

def downsample_with_knn(mesh, steps):
    """
    Downsample a 3D mesh while preserving color information using a K-nearest neighbors (KNN) approach.

    Args:
        mesh (trimesh.Trimesh): The input 3D mesh with color information.
        steps (int): The number of downsampling steps.

    Returns:
        trimesh.Trimesh: The downsampled mesh with predicted colors.
    """
    # Step 1: Train a KNN model on how the input mesh is colored.

    # Get the vertices of the original mesh
    original_mesh_vertices = mesh.vertices

    # Assuming color information is stored in vertex_colors
    original_mesh_colors = mesh.visual.vertex_colors

    # Create a KNN classifier with 3 neighbors
    neigh = KNeighborsClassifier(n_neighbors=3)

    # Train the KNN model on the original mesh's vertices and their colors
    neigh.fit(original_mesh_vertices, original_mesh_colors)

    # Step 2: Clone the original mesh
    mesh2 = mesh.copy()

    # Step 3: Perform mesh simplification to reduce the face count
    mesh2 = simplify_mesh(mesh2, steps)

    # Step 4: Get the vertices of the downsampled mesh
    downsampled_mesh_vertices = mesh2.vertices

    # Step 5: Predict color labels for the downsampled mesh using the KNN model
    predicted_labels = neigh.predict(downsampled_mesh_vertices)

    # Step 6: Add the predicted labels to the downsampled mesh
    mesh2.visual.vertex_colors = predicted_labels

    return mesh2

def initialize_nonrigid(X, Y, beta, lambda_, w):
    """
    This function initializes the parameters to be used in the CPD algorithm.

    X : ndarray
        Source point cloud.
    Y : ndarray
        Target point cloud.
    beta : float
        Gaussian filter standard deviation.
    lambda_ : float
        Weight of the regularization term.
    w : float
        Weight for the uniform distribution in the EM algorithm.

    Returns
    -------
    W : ndarray
        Weight matrix for transforming each point.
    G : ndarray
        Gaussian Affinity matrix.
    X_hat : ndarray
        Initial transformed source point cloud.
    sigma2 : float
        Initial estimate of the variance.
    """
    N, D = X.shape
    M = Y.shape[0]

    # Compute initial sigma2
    sigma2 = (cp.sum(cp.sum(cp.square(X - cp.mean(X, axis=0)))) + cp.sum(cp.sum(cp.square(Y - cp.mean(Y, axis=0))))) / (D * (M + N))
    # The initial estimate of the variance is the sum of variances of X and Y.

    # Initialize W
    W = cp.zeros((M, D))
    # W is initialized to zero, meaning that the initial transformation is the identity.

    # Create Gaussian affinity matrix G
    diff = X[:, None] - X
    G = cp.exp(-cp.sum(diff**2, axis=2) / (2 * beta ** 2))
    # G is computed as the Gaussian of the pairwise distances in X.

    # X_hat is initially X
    X_hat = X
    # The transformed source point cloud starts as a copy of the original.

    return W, G, X_hat, sigma2

def calculate_P_nonrigid(X, Y, sigma2, beta, w):
    """
    This function calculates the matrix P for the CPD algorithm.

    X : ndarray
        Transformed source point cloud.
    Y : ndarray
        Target point cloud.
    sigma2 : float
        Estimate of the variance.
    beta : float
        Gaussian filter standard deviation.
    w : float
        Weight for the uniform distribution in the EM algorithm.

    Returns
    -------
    P : ndarray
        The matrix P.
    """
    N, D = X.shape
    M = Y.shape[0]
    P = cp.empty((N, M))

    diff = X[:, None] - Y
    P = cp.exp(-cp.sum(diff**2, axis=2) / (2 * sigma2 ** 2)) / (sigma2 ** D * (2 * cp.pi) ** (D / 2))
    # P is computed as a Gaussian of the pairwise distances, scaled by the estimated variance.

    cP = w / ((1 - w) * (2 * cp.pi * beta) ** (D / 2))
    den = cp.sum(P, axis=0)
    den = den + cP
    P = cp.divide(P, den[:, cp.newaxis])
    # P is normalized to sum to 1, to form a valid probability distribution.

    return P

def maximize_nonrigid(P, X, Y, G, beta, lambda_):
    """
    This function maximizes the parameters for the CPD algorithm.

    P : ndarray
        The matrix P.
    X : ndarray
        Transformed source point cloud.
    Y : ndarray
        Target point cloud.
    G : ndarray
        Gaussian Affinity matrix.
    beta : float
        Gaussian filter standard deviation.
    lambda_ : float
        Weight of the regularization term.

    Returns
    -------
    W : ndarray
        The updated weight matrix.
    sigma2 : float
        The updated estimate of the variance.
    """
    N, D = X.shape
    M = Y.shape[0]

    # Compute mu_x and mu_y
    ones_N = cp.ones(N)
    Pt1 = cp.dot(P.T, ones_N)
    Px = cp.dot(P.T, X)
    mu_x = cp.sum(Px, axis=0) / cp.sum(Pt1, axis=0)
    mu_y = cp.mean(Y, axis=0)
    # mu_x and mu_y are the centroids of the source and target point clouds, respectively, weighted by the probabilities in P.

    # Compute X_hat and Y_hat
    X_hat = X - mu_x
    Y_hat = Y - mu_y
    # X_hat and Y_hat are the source and target point clouds, translated to have zero centroid.

    P_diag = cp.zeros((M, M))
    cp.fill_diagonal(P_diag, Pt1)
    # P_diag is the diagonal matrix of column sums of P.

    # Compute C
    C = cp.dot(G.T, cp.dot(P_diag, G)) + lambda_ * beta * cp.eye(G.shape[0])
    # C is a regularized version of a weighted covariance matrix of the transformed point cloud.

    # Compute W
    W = cp.dot(cp.linalg.inv(C), Px - cp.dot(G.T, cp.dot(P_diag, X_hat)))
    # W is the solution of a linear system that minimizes the objective function.

    # Compute sigma2
    sigma2 = (cp.trace(X.T.dot(P_diag).dot(X)) + cp.trace(W.T.dot(G).dot(W)) - 2 * cp.trace(Px.T.dot(W))) / (N * D)
    # sigma2 is the updated estimate of the variance, based on the current transformation.

    return W, sigma2

def fit_nonrigid(X, Y, beta=2, lambda_=2, w=0.1, max_iterations=100, tol=1e-5):
    """
    This function implements the non-rigid variant of the Coherent Point Drift (CPD) algorithm.

    X : ndarray
        Source point cloud.
    Y : ndarray
        Target point cloud.
    beta : float
        Gaussian filter standard deviation.
    lambda_ : float
        Weight of the regularization term.
    w : float
        Weight for the uniform distribution in the EM algorithm.
    max_iterations : int
        Maximum number of iterations.
    tol : float
        Tolerance for stopping criterion.

    Returns
    -------
    X_hat : ndarray
        The final transformed source point cloud.
    """
    N, D = X.shape
    M = Y.shape[0]
    W, G, X_hat, sigma2 = initialize_nonrigid(X, Y, beta, lambda_, w)

    for i in range(max_iterations):
        P = calculate_P_nonrigid(X_hat + cp.dot(G, W), Y, sigma2, beta, w)
        W_new, sigma2_new = maximize_nonrigid(P, X_hat, Y, G, beta, lambda_)
        T = X + cp.dot(G, W_new)
        # T is the transformed point cloud after applying the current estimated transformation.

        if cp.linalg.norm(W_new - W) < tol:
            break
        # If the change in W is small, we have converged and can stop early.

        W, sigma2 = W_new, sigma2_new
        # Update the transformation parameters for the next iteration.

        X_hat = T
        # X_hat is updated to be the currently estimated transformed point cloud.

    return X_hat

def get_rmse(deformed_mesh, aligned_mesh):
    return cp.sqrt(cp.mean((cp.asarray(deformed_mesh.vertices) - cp.asarray(aligned_mesh.vertices))**2))

def align_mesh(original_mesh, original_deformed_mesh, threshold=0.01, max_downsample=5):
    rmse = float('inf')
    for downsample_level in range(max_downsample, -1, -1):
        print(f"Downsample level: {downsample_level}")
        # Downsample the original mesh
        intra_mesh = original_mesh.copy()
        intra_deformed_mesh = original_deformed_mesh.copy()

        intra_mesh = simplify_mesh(intra_mesh, downsample_level) # Adjust voxel_size as needed
        intra_deformed_mesh = simplify_mesh(intra_deformed_mesh, downsample_level) # Downsample deformed_mesh too

        # Transfer your data to the GPU
        mesh_gpu = cp.asarray(intra_mesh.vertices)
        deformed_mesh_gpu = cp.asarray(intra_deformed_mesh.vertices)
        try:
          # Call the fit function
            aligned_vertices_gpu = fit_nonrigid(mesh_gpu, deformed_mesh_gpu)
        except ValueError:
          #Skip axis mismatch error stemming from simplify
            print("Non conforming downsampling level")
            continue
        # Transfer the aligned vertices back to the CPU
        aligned_vertices = cp.asnumpy(aligned_vertices_gpu)

        # Apply the transformation to deformed_mesh
        aligned_mesh = deformed_mesh.copy()
        aligned_mesh.vertices = aligned_vertices

        # Calculate the RMSE
        rmse = get_rmse(intra_deformed_mesh, aligned_mesh)
        print(f"RMSE at downsampling level {downsample_level}: {rmse}")

        if rmse <= threshold:
            print(f"Final RMSE: {rmse}")
            return aligned_mesh

    print("Stopped as downsampling level 0 reached.")
    return None

In [9]:
# Apply the modified downsampling function with a specified number of steps
downsampled_mesh = downsample_with_knn(mesh, 5)
downsampled_mesh.show()

## Validation

Threshold should be changed in based on the quality of input image, example: for bunny res 4 it shouldnt be that low

In [21]:
aligned_mesh = align_mesh(mesh, deformed_mesh)
if aligned_mesh is not None:
    aligned_mesh.show()

Downsample level: 5


AttributeError: ignored

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import trimesh

# Load meshes and create segmentation masks
file_names = [
    '/content/AMS_izziv/aneurysms/ru_22_CTA_PT00014_20200906.obj',
    '/content/AMS_izziv/aneurysms/ru_23_CTA_PT00015_20200908.obj',
    '/content/AMS_izziv/aneurysms/ru_24_CTA_PT00016_20200925.obj',
    '/content/AMS_izziv/aneurysms/ru_25_CTA_PT00017_20201222.obj',
    '/content/AMS_izziv/aneurysms/ru_28_CTA_PT00019_20200307.obj'
    #'/content/AMS_izziv/aneurysms/ru_29_CTA_PT00020_20200410.obj'
]

downsampled_meshes = []
segmentation_masks = []

# Function to threshold and binarize colors
def binarize_colors(colors):
    # Get R channel
    r = colors[:, 0]

    # Threshold at 50 (set all values below 50 to 0 and all values above or equal to 50 to 1)
    r_binarized = np.where(r < 50, 0, 1)

    return r_binarized

# Iterate over all file names
for file_name in file_names:
    # Load the original mesh
    original_mesh = trimesh.load_mesh(file_name)

    # Binarize R channel colors
    r_binarized = binarize_colors(original_mesh.visual.vertex_colors)

    # Add mesh vertices and binarized colors to respective lists
    downsampled_meshes.append(original_mesh.vertices)
    segmentation_masks.append(r_binarized)

# Build a simple Neural Network model
model = models.Sequential([
    layers.Dense(256, activation='tanh', input_shape=(3,)),
    layers.Dense(128, activation='tanh'),
    layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
for mesh, mask in zip(downsampled_meshes, segmentation_masks):
    # Flatten the mesh vertices data and prepare it for the Neural Network
    X = mesh.reshape(mesh.shape[0], -1)

    # Convert segmentation masks to categorical (one-hot encoding)
    Y = to_categorical(mask)

    # Train the model on this mesh
    model.fit(X, Y, epochs=10, batch_size=32)

Epoch 1/10
157/157 [==============================] - 2s 4ms/step - loss: 0.2090 - accuracy: 0.9308
Epoch 2/10
157/157 [==============================] - 1s 3ms/step - loss: 0.0949 - accuracy: 0.9622
Epoch 3/10
157/157 [==============================] - 1s 3ms/step - loss: 0.0768 - accuracy: 0.9668
Epoch 4/10
157/157 [==============================] - 1s 3ms/step - loss: 0.0812 - accuracy: 0.9672
Epoch 5/10
157/157 [==============================] - 0s 3ms/step - loss: 0.0653 - accuracy: 0.9762
Epoch 6/10
157/157 [==============================] - 1s 3ms/step - loss: 0.0580 - accuracy: 0.9754
Epoch 7/10
157/157 [==============================] - 1s 3ms/step - loss: 0.0541 - accuracy: 0.9794
Epoch 8/10
157/157 [==============================] - 1s 3ms/step - loss: 0.0509 - accuracy: 0.9768
Epoch 9/10
157/157 [==============================] - 1s 3ms/step - loss: 0.0598 - accuracy: 0.9762
Epoch 10/10
157/157 [==============================] - 1s 3ms/step - loss: 0.0641 - accuracy: 0.9732

In [27]:
# Load the test mesh
output_test_mesh = trimesh.load('/content/AMS_izziv/aneurysms/ru_29_CTA_PT00020_20200410.obj')

# Extract the vertices
test_vertices = output_test_mesh.vertices

# Reshape it to match the shape that your model expects
X_test = test_vertices.reshape(test_vertices.shape[0], -1)

# Perform prediction using the model
Y_test_pred = model.predict(X_test)

# Convert the predictions to binary labels
Y_test_pred_binary = np.argmax(Y_test_pred, axis=1)

print(Y_test_pred_binary)
# Create a new mesh that is a copy of the test mesh
view_mesh = output_test_mesh.copy()

# Assign the predicted labels to the vertices of the new mesh
view_mesh.visual.vertex_colors = Y_test_pred_binary
# Map binary labels to RGB colors
colors = np.zeros((len(Y_test_pred_binary), 3))
colors[Y_test_pred_binary == 0] = [255, 0, 0]  # Set label 0 to red
colors[Y_test_pred_binary == 1] = [0, 0, 0]  # Set label 1 to green

# Assign colors to the vertices of the new mesh
view_mesh.visual.vertex_colors = colors

# Visualize the mesh
view_mesh.show()

157/157 [==============================] - 0s 1ms/step
[1 1 1 ... 1 1 1]
